In [2]:
# !pip install -r requirements.txt
date.today()

datetime.date(2019, 11, 27)

In [1]:
import numpy as np 
import pandas as pd 
from cryptography.fernet import Fernet
from datetime import datetime, date, timedelta
from tqdm import tqdm_notebook as tqdm
import os
import pymssql as mssql
import calendar
import time
from multiprocessing import Pool
from functools import partial

import ALL_STOCK_preprocess_function as func
import VWAP_feature_function_rolling_week as func1

In [5]:
end_date = date.today()
d = func.stock_query(date(2019,11,30))

Exception: Data Not Updated

In [2]:
d, r = func.send_query(f'''SELECT [DATE] AS ts,
                                TRY_CAST([OPEN] AS FLOAT) AS [index_open], 
                                TRY_CAST([HIGH] AS FLOAT) AS [index_high],
                                TRY_CAST([LOW] AS FLOAT) AS [index_low],
                                TRY_CAST([CLOSE] AS FLOAT) AS [index_close],
                                TRY_CAST([VOLUME] AS FLOAT) AS index_vol
                        FROM OpenData.dbo.CMONEY_DAILY_CLOSE
                        WHERE 
                            STOCK_ID = 'TWA00' 
                            AND DATE <= '20191031' and DATE >= '20191001'
                        ''')

In [3]:
r

21

In [3]:
end_date = date.today()
d = func.stock_query(end_date)
stock, index, industry_index = d[0], d[1], d[2]
df_list = [group[1] for group in stock.groupby(stock['StockNo'])]
timedf = pd.DataFrame(d[0]['ts'].unique(), columns=['ts'])
timedf = timedf.sort_values(by='ts')
output_list = []

if __name__ == '__main__':
    with Pool(processes=5) as pool:
        for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timedf=timedf, end_date=end_date), df_list), total=len(df_list)), 1):
                if x[0]:
                    output_list.append(x[1])
                else:
                    continue

df = pd.concat(output_list)

df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=5) as pool:
        for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.merge_index, index=index, industry_index=industry_index), df_list), total=len(df_list)), 1):
                output_list.append(x)

df = pd.concat(output_list)

columns_dict = {'lag': ['index_close', 'industry_close', 'close', 'VWAP', 'VWAP_day5'],
           'return': ['index', 'industry', 'close', 'VWAP', 'VWAP_day5'],
           'ratio': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
           'momentum': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
            'moment': ['index_close', 'index_vol', 'industry_close', 'industry_vol', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
            'buyer': ['foreign_buy', 'investment_buy', 'dealer_buy']
           }

df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=5) as pool:
        for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func1.get_technical_indicators, columns_dict=columns_dict), df_list), total=len(df_list)), 1):
            output_list.append(x)


df = pd.concat(output_list, axis=0)    
df_last = df[df.ts.dt.date == end_date]

In [6]:
import json
def read_feature_list(file_path, requirement='whole'):

    assert requirement in ['whole', 'technical', 'buy', 'index']

    d = []
    with open(file_path, 'r') as fp:
        d = json.load(fp)
    
    feature_list = d[requirement]

    return feature_list

feature_list = read_feature_list('D:\\庫存健診開發\\feature_dict.json')

In [8]:
df_last[feature_list].isna().values.any()

False

In [9]:
import requests
import os
def prediction(df):

    df_json = df.to_dict(orient='records')
    x = requests.post('http://128.110.238.61:8080/predict', data = json.dumps({'instances':df_json}))
    result = json.loads(x.text)
    prediction = result.get('predictions')
    scores = prediction[0].get('scores')
    this_result = list(df[['StockNo','ts']].values[0])+[scores[0],scores[1]]

    return this_result    

In [12]:
feature_df = df_last[feature_list]
df_last['ts'] = df_last['ts'].astype(str)

results=[]
for i in tqdm(range(len(feature_df))):
    this_df = feature_df.iloc[[i]]
    result = prediction(this_df)
    results.append(result)

result_df = pd.DataFrame(results, columns=['StockNo','ts','Y_0_score','Y_1_score'])
print(result_df)

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



    StockNo          ts  Y_0_score  Y_1_score
0      1102  2019-11-27   0.399627   0.600373
1      1101  2019-11-27   0.599607   0.400393
2      1103  2019-11-27   0.734911   0.265089
3      1109  2019-11-27   0.669567   0.330433
4      1201  2019-11-27   0.609851   0.390149
..      ...         ...        ...        ...
549    9943  2019-11-27   0.284701   0.715299
550    9945  2019-11-27   0.563494   0.436506
551    9946  2019-11-27   0.405217   0.594783
552    9955  2019-11-27   0.720539   0.279461
553    9958  2019-11-27   0.432235   0.567765

[554 rows x 4 columns]


In [14]:
feature_df.to_csv(f'20191126_feature.csv',index=False)

In [15]:
result_df.to_csv('test_prediction_20191127.csv', index=False)

In [9]:
len(feature_list)

178

In [1]:
import json

In [12]:
whole_list = ['total', 'capital', 'index_open', 'index_high', 'index_low', 
        'index_close', 'index_return', 'index_vol', 
        'industry_return', 'foreign_ratio', 'investment_ratio', 
        'corporation_ratio', 'close_return', 'VWAP_return', 
        'index_VWAP_slope', 'VWAP_day5_return', 'VWAP_day5_return_5',
       'industry_VWAP_slope', 'index_close_slope', 'industry_close_slope',
       'index_close_ratio_day5', 'index_vol_ratio_day5',
       'index_return_ratio_day5', 'industry_close_ratio_day5',
       'industry_vol_ratio_day5', 'industry_return_ratio_day5',
       'VWAP_return_ratio_day5', 'close_return_ratio_day5', 'vol_ratio_day5',
       'VWAP_ratio_day5', 'index_close_ratio_day10', 'index_vol_ratio_day10',
       'index_return_ratio_day10', 'industry_close_ratio_day10',
       'industry_vol_ratio_day10', 'industry_return_ratio_day10',
       'VWAP_return_ratio_day10', 'close_return_ratio_day10',
       'vol_ratio_day10', 'VWAP_ratio_day10', 'index_close_ratio_day15',
       'index_vol_ratio_day15', 'index_return_ratio_day15',
       'industry_close_ratio_day15', 'industry_vol_ratio_day15',
       'industry_return_ratio_day15', 'VWAP_return_ratio_day15',
       'close_return_ratio_day15', 'vol_ratio_day15', 'VWAP_ratio_day15',
       'index_close_momentum_day1', 'index_vol_momentum_day1',
       'index_return_momentum_day1', 'industry_close_momentum_day1',
       'industry_vol_momentum_day1', 'industry_return_momentum_day1',
       'VWAP_return_momentum_day1', 'close_return_momentum_day1',
       'vol_momentum_day1', 'VWAP_momentum_day1', 'index_close_momentum_day2',
       'index_vol_momentum_day2', 'index_return_momentum_day2',
       'industry_close_momentum_day2', 'industry_vol_momentum_day2',
       'industry_return_momentum_day2', 'VWAP_return_momentum_day2',
       'close_return_momentum_day2', 'vol_momentum_day2', 'VWAP_momentum_day2',
       'index_close_momentum_day3', 'index_vol_momentum_day3',
       'index_return_momentum_day3', 'industry_close_momentum_day3',
       'industry_vol_momentum_day3', 'industry_return_momentum_day3',
       'VWAP_return_momentum_day3', 'close_return_momentum_day3',
       'vol_momentum_day3', 'VWAP_momentum_day3',
       'index_close_momentum_day4', 'index_vol_momentum_day4',
       'index_return_momentum_day4', 'industry_close_momentum_day4',
       'industry_vol_momentum_day4', 'industry_return_momentum_day4',
       'VWAP_return_momentum_day4', 'close_return_momentum_day4',
       'vol_momentum_day4', 'VWAP_momentum_day4', 'index_close_momentum_day5',
       'index_vol_momentum_day5', 'index_return_momentum_day5',
       'industry_close_momentum_day5', 'industry_vol_momentum_day5',
       'industry_return_momentum_day5', 'VWAP_return_momentum_day5',
       'close_return_momentum_day5', 'vol_momentum_day5', 'VWAP_momentum_day5',
       'index_close_momentum_day10', 'index_vol_momentum_day10',
       'index_return_momentum_day10', 'industry_close_momentum_day10',
       'industry_vol_momentum_day10', 'industry_return_momentum_day10',
       'VWAP_return_momentum_day10', 'close_return_momentum_day10',
       'vol_momentum_day10', 'VWAP_momentum_day10', 'index_close_momentum_day15',
       'index_vol_momentum_day15', 'index_return_momentum_day15',
       'industry_close_momentum_day15', 'industry_vol_momentum_day15',
       'industry_return_momentum_day15', 'VWAP_return_momentum_day15',
       'close_return_momentum_day15', 'vol_momentum_day15',
       'VWAP_momentum_day15', 'vol_VWAP_corr', 'vol_volatile_corr',
       'index_vol_close_corr', 'index_vol_volatile_corr',
       'industry_vol_close_corr', 'industry_vol_volatile_corr',
       'index_close_skew', 'index_close_kurtosis', 'index_vol_skew',
       'index_vol_kurtosis', 'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'vol_skew', 'vol_kurtosis', 'VWAP_skew',
       'VWAP_kurtosis', 'VWAP_day5_skew', 'VWAP_day5_kurtosis',
       'foreign_buy_ratio', 'foreign_buy_day3_ratio', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_buy_indicator',
       'foreign_buy_sell_indicator', 'investment_buy_ratio',
       'investment_buy_day3_ratio', 'investment_buy_week_ratio',
       'investment_buy_2week_ratio', 'investment_buy_3week_ratio',
       'investment_buy_continuous', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'dealer_buy_ratio',
       'dealer_buy_day3_ratio', 'dealer_buy_week_ratio',
       'dealer_buy_2week_ratio', 'dealer_buy_3week_ratio',
       'dealer_buy_continuous', 'dealer_buy_buy_day_ratio',
       'dealer_buy_sell_day_ratio', 'index_pvt', 'industry_pvt', 'foreign_pvt',
       'investment_pvt', 'dealer_pvt', 'pvt', 'RSI_15', 'SOk5_1', 'SOd3',
       'SOk15_5', 'SOd5', 'industry', 'ts', 'StockNo']

technical = ['total', 'capital', 'close_return', 'VWAP_return', 'VWAP_day5_return', 'VWAP_day5_return_5',
            'VWAP_return_ratio_day5', 'close_return_ratio_day5', 'vol_ratio_day5',
           'VWAP_ratio_day5', 'VWAP_return_ratio_day10', 'close_return_ratio_day10',
       'vol_ratio_day10', 'VWAP_ratio_day10', 'VWAP_return_ratio_day15',
       'close_return_ratio_day15', 'vol_ratio_day15', 'VWAP_ratio_day15', 'VWAP_return_momentum_day1', 
        'close_return_momentum_day1', 'vol_momentum_day1', 'VWAP_momentum_day1', 
        'VWAP_return_momentum_day2', 'close_return_momentum_day2', 'vol_momentum_day2', 'VWAP_momentum_day2',
        'VWAP_return_momentum_day3', 'close_return_momentum_day3',
       'vol_momentum_day3', 'VWAP_momentum_day3', 'VWAP_return_momentum_day4', 'close_return_momentum_day4',
       'vol_momentum_day4', 'VWAP_momentum_day4', 'VWAP_return_momentum_day5',
       'close_return_momentum_day5', 'vol_momentum_day5', 'VWAP_momentum_day5',
        'VWAP_return_momentum_day10', 'close_return_momentum_day10',
       'vol_momentum_day10', 'VWAP_momentum_day10', 'VWAP_return_momentum_day15',
       'close_return_momentum_day15', 'vol_momentum_day15',
       'VWAP_momentum_day15', 'vol_VWAP_corr', 'vol_volatile_corr',
        'vol_skew', 'vol_kurtosis', 'VWAP_skew',
       'VWAP_kurtosis', 'VWAP_day5_skew', 'VWAP_day5_kurtosis', 'pvt', 'RSI_15', 'SOk5_1', 'SOd3',
       'SOk15_5', 'SOd5', 'industry', 'ts', 'StockNo']

buy = ['total', 'capital', 'foreign_ratio', 'investment_ratio', 
        'corporation_ratio', 'foreign_buy_ratio', 'foreign_buy_day3_ratio', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_buy_indicator',
       'foreign_buy_sell_indicator', 'investment_buy_ratio',
       'investment_buy_day3_ratio', 'investment_buy_week_ratio',
       'investment_buy_2week_ratio', 'investment_buy_3week_ratio',
       'investment_buy_continuous', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'dealer_buy_ratio',
       'dealer_buy_day3_ratio', 'dealer_buy_week_ratio',
       'dealer_buy_2week_ratio', 'dealer_buy_3week_ratio',
       'dealer_buy_continuous', 'dealer_buy_buy_day_ratio',
       'dealer_buy_sell_day_ratio', 'foreign_pvt',
       'investment_pvt', 'dealer_pvt', 'industry', 'ts', 'StockNo']

index = ['total', 'capital', 'index_open', 'index_high', 'index_low', 
        'index_close', 'index_return', 'index_vol', 
        'industry_return', 'index_VWAP_slope', 'industry_VWAP_slope', 'index_close_slope', 'industry_close_slope',
        'index_close_ratio_day5', 'index_vol_ratio_day5',
       'index_return_ratio_day5', 'industry_close_ratio_day5',
       'industry_vol_ratio_day5', 'industry_return_ratio_day5', 
         'index_close_ratio_day10', 'index_vol_ratio_day10',
       'index_return_ratio_day10', 'industry_close_ratio_day10',
       'industry_vol_ratio_day10', 'industry_return_ratio_day10',
        'index_close_ratio_day15','index_vol_ratio_day15', 
        'index_return_ratio_day15', 'industry_close_ratio_day15', 
        'industry_vol_ratio_day15', 'industry_return_ratio_day15', 
        'index_close_momentum_day1', 'index_vol_momentum_day1',
       'index_return_momentum_day1', 'industry_close_momentum_day1',
       'industry_vol_momentum_day1', 'industry_return_momentum_day1', 'index_close_momentum_day2',
       'index_vol_momentum_day2', 'index_return_momentum_day2',
       'industry_close_momentum_day2', 'industry_vol_momentum_day2',
       'industry_return_momentum_day2', 'index_close_momentum_day3', 'index_vol_momentum_day3',
       'index_return_momentum_day3', 'industry_close_momentum_day3',
       'industry_vol_momentum_day3', 'industry_return_momentum_day3',
        'index_close_momentum_day4', 'index_vol_momentum_day4',
       'index_return_momentum_day4', 'industry_close_momentum_day4',
       'industry_vol_momentum_day4', 'industry_return_momentum_day4', 'index_close_momentum_day5',
       'index_vol_momentum_day5', 'index_return_momentum_day5',
       'industry_close_momentum_day5', 'industry_vol_momentum_day5',
       'industry_return_momentum_day5', 'index_close_momentum_day10', 'index_vol_momentum_day10',
       'index_return_momentum_day10', 'industry_close_momentum_day10',
       'industry_vol_momentum_day10', 'industry_return_momentum_day10', 'index_close_momentum_day15',
       'index_vol_momentum_day15', 'index_return_momentum_day15',
       'industry_close_momentum_day15', 'industry_vol_momentum_day15',
       'industry_return_momentum_day15', 'index_vol_close_corr', 'index_vol_volatile_corr',
       'industry_vol_close_corr', 'industry_vol_volatile_corr',
       'index_close_skew', 'index_close_kurtosis', 'index_vol_skew',
       'index_vol_kurtosis', 'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'index_pvt', 'industry_pvt', 
        'industry', 'ts', 'StockNo']

In [7]:
data = {}
data['whole'] = feature_list
data['technical'] = technical
data['buy'] = buy
data['index'] = index

In [8]:
data

{'whole': ['total',
  'capital',
  'index_open',
  'index_high',
  'index_low',
  'index_close',
  'index_return',
  'index_vol',
  'industry_return',
  'foreign_ratio',
  'investment_ratio',
  'corporation_ratio',
  'close_return',
  'VWAP_return',
  'index_VWAP_slope',
  'VWAP_day5_return',
  'VWAP_day5_return_5',
  'industry_VWAP_slope',
  'index_close_slope',
  'industry_close_slope',
  'index_close_ratio_day5',
  'index_vol_ratio_day5',
  'index_return_ratio_day5',
  'industry_close_ratio_day5',
  'industry_vol_ratio_day5',
  'industry_return_ratio_day5',
  'VWAP_return_ratio_day5',
  'close_return_ratio_day5',
  'vol_ratio_day5',
  'VWAP_ratio_day5',
  'index_close_ratio_day10',
  'index_vol_ratio_day10',
  'index_return_ratio_day10',
  'industry_close_ratio_day10',
  'industry_vol_ratio_day10',
  'industry_return_ratio_day10',
  'VWAP_return_ratio_day10',
  'close_return_ratio_day10',
  'vol_ratio_day10',
  'VWAP_ratio_day10',
  'index_close_ratio_day15',
  'index_vol_ratio_day1

In [10]:
with open('D:\\庫存健診開發\\feature_dict.json', 'w') as fp:
    json.dump(data, fp)

In [13]:
len(whole_list) == len(feature_list)

True

In [14]:
with open('D:\\庫存健診開發\\feature_dict.json', 'r') as fp:
    d = json.load(fp)
d['whole']

['total',
 'capital',
 'index_open',
 'index_high',
 'index_low',
 'index_close',
 'index_return',
 'index_vol',
 'industry_return',
 'foreign_ratio',
 'investment_ratio',
 'corporation_ratio',
 'close_return',
 'VWAP_return',
 'index_VWAP_slope',
 'VWAP_day5_return',
 'VWAP_day5_return_5',
 'industry_VWAP_slope',
 'index_close_slope',
 'industry_close_slope',
 'index_close_ratio_day5',
 'index_vol_ratio_day5',
 'index_return_ratio_day5',
 'industry_close_ratio_day5',
 'industry_vol_ratio_day5',
 'industry_return_ratio_day5',
 'VWAP_return_ratio_day5',
 'close_return_ratio_day5',
 'vol_ratio_day5',
 'VWAP_ratio_day5',
 'index_close_ratio_day10',
 'index_vol_ratio_day10',
 'index_return_ratio_day10',
 'industry_close_ratio_day10',
 'industry_vol_ratio_day10',
 'industry_return_ratio_day10',
 'VWAP_return_ratio_day10',
 'close_return_ratio_day10',
 'vol_ratio_day10',
 'VWAP_ratio_day10',
 'index_close_ratio_day15',
 'index_vol_ratio_day15',
 'index_return_ratio_day15',
 'industry_close_r

In [3]:
d = func.stock_query(date(2019,11,27))
stock, index, industry_index = d[0], d[1], d[2]

In [5]:
stock['ts'].max()

Timestamp('2019-11-27 00:00:00')

In [7]:
df_last = stock[stock.ts.dt.date == date(2019,11,27)]

In [8]:
df_last['ts'].unique()

array(['2019-11-27T00:00:00.000000000'], dtype='datetime64[ns]')